In [346]:
# import os
# os.chdir("../")

In [347]:
import pandas as pd

from clean.post_officer_history import clean

In [348]:
df = clean()

standardize_from_lookup_table: unmatched sequences:
  {"orleans parish sheriff's academy\nnopd municipal training academy\n\nfirearm qualifications\nrange / academy\nnopd municipal training academy\norleans parish sheriff's academy\nplaquemines parish so range", 'reserve 4/1/2001 5/1/2002', 'retired 2/7/1988 7/1/2016', 'bossier parish so retired — 6/162010s72172022', 'nopd municipal training academy\nla state police academy', 'reserve tat/', '9/28/1976 8/6/1987'}


c:\Users\PC\Desktop\launch\processing\clean\post_officer_history.py:313: FutureWarning: The default value of regex will change from True to False in a future version.
  df.agency.str.strip()


In [349]:
"""
Preview
"""
df.head(10)

,history_id,agency,last_name,first_name,middle_name,left_reason,hire_date,left_date,employment_status,uid,switched_job,source_agency
0,0,jefferson-so,Alvarado,Julio,C,NaN,2/28/2005,,full-time,cd9f1f562357f0f3307603cafda8d032,True,post
1,0,new-orleans-so,Alvarado,Julio,C,NaN,5/17/2002,,full-time,1554036f19bab5d19a59a508d773984e,True,post
3,1,new-orleans-pd,Vaught,Ryan,Clinton,NaN,10/7/2004,9/17/2016,full-time,3df20fb350cbf9b1e748c0f671dcf008,True,post
4,1,jefferson-so,Vaught,Ryan,Clinton,NaN,9/19/2016,,reserve,5242a69e525917a838f93af4196ec656,True,post
5,2,harahan-pd,Mehrtens,Steven,Allen,NaN,9/1/2011,,full-time,8c342854dcfcd41aad83ddcefe72f104,True,post
6,2,jefferson-so,Mehrtens,Steven,Allen,NaN,10/3/2016,,full-time,74f6f774e80a9c7f3d7c3a6555cf8217,True,post
8,4,jefferson-so,Gaudet,Myron,A,NaN,8/14/1992,,full-time,50e1899068d40e08d23b920a9321ea60,False,post
9,5,new-orleans-so,Dowling,Keith,Edward,NaN,7/1/1993,4/1/1994,full-time,95e86285a377f1f40a0efad49209e74e,True,post
10,5,st-bernard-so,Dowling,Keith,Edward,NaN,1/12/1998,2/1/2001,full-time,22b2cfbe116d149542e913d226a1192f,True,post
11,5,st-tammany-so,Dowling,Keith,Edward,NaN,11/15/2002,,full-time,029bfb6d971f9054825a5c2ca848e716,True,post


In [350]:
"""
Filter out officers who haven't switched employers
"""
df = df[df.switched_job == True]

In [351]:
"""
Act 272 of 2017 requires all law enforcement agencies to report changes to an officer's employment status
---------------------------------------------------------------------------------------------------------
Filter out data prior to 2018
Filter out officers who retired or are deceased 
Fill empty rows with n/a values
"""
def filter_years(df):
    year = df.left_date.str.extract(r"(\w{4})")
    df.loc[:, "left_year"] = year[0]

    df = df[df.left_year.isin(["2018", "2019", "2020", "2021", "2022"])]
    return df

def filter_employment_status(df):
    df = df[~((df.employment_status == "retired"))]
    df = df[~((df.employment_status == "deceased"))]
    return df

def fill_na_values(df):
    df.loc[:, "left_reason"] = df.left_reason.fillna("n/a")
    return df

df = df.pipe(filter_years).pipe(filter_employment_status).pipe(fill_na_values)

In [352]:
"""
Between 2018 - 2022 the following number of officers had a change in employment status
"""
df.history_id.nunique()

140

In [353]:
"""
Officers who switched employers in these years moved between the following number of agencies
"""
df.agency.nunique()

81

In [354]:
"""
43% of the officers who switched employers between 2018 - 2022 have no reported reason for a change in employment status, despite the legal requirement on agenccies to provide one.
"""
df.left_reason.value_counts(normalize=True)

voluntary resignation      0.465116
n/a                        0.430233
termination                0.058140
resignation                0.040698
involuntary resignation    0.005814
Name: left_reason, dtype: float64

In [355]:
"""
Drop rows missing data to compare left reasons
"""
def drop_na_rows(df):
    df.loc[:, "left_reason"] = df.left_reason.str.replace(r"^n\/a$", "", regex=True)
    return df[~((df.left_reason == ""))]

df = df.pipe(drop_na_rows)

In [356]:
"""
Of the 43%, the most commonly reported left_reason is voluntary resignation, followed by termination, resignation and involuntary resignation.
"""
df.left_reason.value_counts(normalize=True)

voluntary resignation      0.816327
termination                0.102041
resignation                0.071429
involuntary resignation    0.010204
Name: left_reason, dtype: float64